# 核心能力提升班商业智能方向 004期 Action2

## Kaggle 员工离职率预测

https://www.kaggle.com/c/bi-attrition-prediction?rvi=1

In [1]:
# 引包
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [2]:
# 加载文件
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")
train_data.head()

,user_id,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1374,58,No,Travel_Rarely,605,Sales,21,3,Life Sciences,1,...,3,80,1,29,2,2,1,0,0,0
1,1092,45,No,Travel_Rarely,950,Research & Development,28,3,Technical Degree,1,...,4,80,1,8,3,3,5,4,0,3
2,768,40,No,Travel_Rarely,300,Sales,26,3,Marketing,1,...,2,80,1,8,3,2,7,7,7,5
3,569,36,No,Non-Travel,1434,Sales,8,4,Life Sciences,1,...,2,80,0,10,1,3,10,7,0,9
4,911,25,Yes,Travel_Frequently,599,Sales,24,1,Life Sciences,1,...,4,80,0,1,4,3,1,0,1,0


In [3]:
test_data.head()

,user_id,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,442,36,Non-Travel,635,Sales,10,4,Medical,1,592,...,4,80,0,10,3,2,10,3,9,7
1,1091,33,Travel_Rarely,575,Research & Development,25,3,Life Sciences,1,1545,...,4,80,0,5,2,3,5,3,0,2
2,981,35,Travel_Frequently,662,Sales,18,4,Marketing,1,1380,...,3,80,1,5,0,2,4,2,3,2
3,785,40,Travel_Rarely,1492,Research & Development,20,4,Technical Degree,1,1092,...,4,80,1,14,6,3,11,10,11,1
4,1332,29,Travel_Frequently,459,Research & Development,24,2,Life Sciences,1,1868,...,2,80,0,1,3,2,1,0,1,0


In [4]:
# Label 处理
train_y = train_data["Attrition"]
train_y

0        No
1        No
2        No
3        No
4       Yes
       ... 
1171     No
1172     No
1173     No
1174     No
1175     No
Name: Attrition, Length: 1176, dtype: object

In [5]:
train_y = LabelEncoder().fit_transform(train_y).reshape(-1,1)
train_y.shape

(1176, 1)

In [6]:
# 分离离散值
DV = ["BusinessTravel", "Department", "EducationField", "Gender", "JobRole", "MaritalStatus", "Over18", "OverTime", "StockOptionLevel"]
train_DV = train_data[DV]
train_DV.head()

,BusinessTravel,Department,EducationField,Gender,JobRole,MaritalStatus,Over18,OverTime,StockOptionLevel
0,Travel_Rarely,Sales,Life Sciences,Female,Manager,Married,Y,Yes,1
1,Travel_Rarely,Research & Development,Technical Degree,Male,Research Scientist,Married,Y,No,1
2,Travel_Rarely,Sales,Marketing,Male,Sales Executive,Married,Y,No,1
3,Non-Travel,Sales,Life Sciences,Male,Sales Executive,Single,Y,No,0
4,Travel_Frequently,Sales,Life Sciences,Male,Sales Representative,Single,Y,Yes,0


In [7]:
# 连续值
CV = ["Age", "DailyRate", "Education", "EmployeeNumber", "EnvironmentSatisfaction", "JobInvolvement",
      "JobLevel", "JobSatisfaction", "MonthlyIncome", "NumCompaniesWorked", "PercentSalaryHike", "PerformanceRating",
     "RelationshipSatisfaction", "StandardHours", "TotalWorkingYears", "TrainingTimesLastYear", "WorkLifeBalance", 
      "YearsAtCompany", "YearsInCurrentRole", "YearsSinceLastPromotion", "YearsWithCurrManager",'DistanceFromHome',
      'EmployeeCount', 'HourlyRate', 'MonthlyRate']
train_CV = train_data[CV]
train_CV.head()

,Age,DailyRate,Education,EmployeeNumber,EnvironmentSatisfaction,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,...,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,DistanceFromHome,EmployeeCount,HourlyRate,MonthlyRate
0,58,605,3,1938,4,3,4,4,17875,4,...,2,2,1,0,0,0,21,1,72,11761
1,45,950,3,1546,4,3,1,4,2132,4,...,3,3,5,4,0,3,28,1,97,4585
2,40,300,3,1066,3,3,2,1,8396,1,...,3,2,7,7,7,5,26,1,74,22217
3,36,1434,4,789,1,2,3,1,7587,1,...,1,3,10,7,0,9,8,1,76,14229
4,25,599,1,1273,3,1,1,4,1118,1,...,4,3,1,0,1,0,24,1,73,8040


In [8]:
# 处理离散值
# LabelEncoder
train_DV_label = train_DV.apply(LabelEncoder().fit_transform)
#train_DV_label.head()
train_DV_label = np.array(train_DV_label)
train_DV_label.shape

(1176, 9)

In [9]:
# OneHotEncoder
train_DV_onehot = OneHotEncoder().fit_transform(train_DV).toarray()
train_DV_onehot.shape

(1176, 33)

In [10]:
# 处理连续值
# 规范化
train_CV_SS  = StandardScaler().fit_transform(train_CV)
train_CV_SS.shape

(1176, 25)

In [11]:
train_x = np.concatenate([train_DV_label, train_CV_SS], axis = 1)
train_x.shape

(1176, 34)

In [12]:
# 数据预处理方法
def data_preprocessing(data, CV, DV, Encoder = "Label", SS = True):
    train_DV = data[DV]
    train_CV = data[CV]
    if Encoder == "Label":
        train_DV = np.array(train_DV.apply(LabelEncoder().fit_transform))
    elif Encoder == "OneHot":
        train_DV = OneHotEncoder().fit_transform(train_DV).toarray()
    if SS:
        train_CV = StandardScaler().fit_transform(train_CV)
    return np.concatenate([train_DV, train_CV], axis = 1)

In [13]:
test_x = data_preprocessing(test_data, CV, DV, "Label")
test_x.shape

(294, 34)

In [14]:
import sklearn.pipeline as pl
import sklearn.preprocessing as sp
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import roc_auc_score

In [15]:
# 对不同分类器进行效果验证
def train_and_valid(train_x, train_y, Classifier, n_splits=10):
    # 产生训练集和验证集
    # K折交叉验证法
    KF = KFold(n_splits)
    KF.get_n_splits(train_x)
    roc_score = []
    for train_index,valid_index in KF.split(train_x):
        x_train,x_valid=train_x[train_index],train_x[valid_index]
        y_train,y_valid=train_y[train_index],train_y[valid_index]
        Classifier.fit(x_train, y_train)
        y_pred = Classifier.predict(x_valid)
        roc_score.append(roc_auc_score(y_valid, y_pred))
    return np.mean(roc_score)

In [16]:
# CART 分类器
train_and_valid(train_x, train_y, DecisionTreeClassifier())

0.6125003066989996

In [17]:
# 逻辑回归分类器
train_and_valid(train_x, train_y.ravel(), LogisticRegression(max_iter = 1000))

0.638399642825527

In [18]:
# 逻辑回归分类器
train_and_valid(train_x, train_y.ravel(), LogisticRegression(max_iter = 1000, penalty="none"))

0.6486567578227408

In [19]:
# 线性回归
train_and_valid(train_x, train_y.ravel(), LinearRegression())

0.8049314032319007

In [20]:
model = pl.make_pipeline(
        sp.PolynomialFeatures(2),  # 多项式特征拓展器
        LinearRegression()  # 线性回归器
)
train_and_valid(train_x, train_y.ravel(), model)

0.7440530864217498

In [21]:
train_x = data_preprocessing(train_data, CV, DV, "OneHot")
test_x = data_preprocessing(test_data, CV, DV, "OneHot")

In [22]:
# CART 分类器 + OneHot
train_and_valid(train_x, train_y, DecisionTreeClassifier())

0.6041959017643119

In [23]:
# 逻辑回归分类器 + OneHot
train_and_valid(train_x, train_y.ravel(), LogisticRegression(max_iter = 1000))

0.688945909867424

In [ ]:
# 逻辑回归分类器 + OneHot
train_and_valid(train_x, train_y.ravel(), LogisticRegression(max_iter = 1000, penalty="none"))

In [ ]:
# 线性回归 + OneHot
train_and_valid(train_x, train_y.ravel(), LinearRegression())

In [ ]:
model = pl.make_pipeline(
        sp.PolynomialFeatures(2),  # 多项式特征拓展器
        LinearRegression()  # 线性回归器
)
train_and_valid(train_x, train_y.ravel(), model)

In [ ]:
# 选择OneHot 编码 + 线性回归
Classifier = LinearRegression().fit(train_x, train_y.ravel())
pred_y = Classifier.predict(test_x)
pred_y

In [ ]:
result = {
    "user_id": test_data["user_id"].values,
    "Attrition": pred_y
}
DF = pd.DataFrame(result)
DF

In [ ]:
DF.to_csv("result.csv",index=0)